In [1]:
pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 857.8/857.8 kB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 kB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.1/318.1 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 35.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 4.3 MB

In [2]:
#Importation des biblothèque necesaire
import torch
from transformers import BertModel, BertTokenizer, BertConfig
from torch.utils.data import DataLoader, Dataset
import pandas as pd
import gradio as gr
from google.colab import drive
drive.mount('/content/drive')
import sys
sys.path.append('/content/drive/My Drive/Classification')
from Classification_langue import BERTSequenceClassifier

# Chargement des fichiers
data_path = '/content/drive/My Drive/Classification/bert_sequence_classifier.pth'
data = pd.read_csv('/content/drive/My Drive/Classification/contradictory-my-dear.csv')

#Actualisation des labels

dico_mapping = {}

categories_uniques = data['language'].unique()
for i, categorie in enumerate(categories_uniques):
    dico_mapping[categorie] = i

data['label'] = data['language'].map(dico_mapping)

# Vérifier si le GPU est disponible, sinon utiliser le CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Chargement du tokenizer et du modèle entraîné
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BERTSequenceClassifier(num_labels=len(data['label'].unique())).to(device)
model.load_state_dict(torch.load(data_path))
model.eval()

# Définition de la fonction de prédiction de la langue
def predict_language(text):
    if text.lower() == "effacer":
        return ""

    input_ids = tokenizer.encode(text, add_special_tokens=True, max_length=128, truncation=True, padding='max_length', return_tensors='pt').to(device)
    attention_mask = input_ids != 0
    outputs = model(input_ids, attention_mask)
    predicted_label = torch.argmax(outputs[0]).item()

    return data['language'].unique()[predicted_label]

# Création de l'interface Gradio pour l'API
Interface = gr.Interface(fn=predict_language, inputs="text", outputs="text", title="API de prédiction de langue", description="Entrez le texte à prédire")

# Lancement de l'API
Interface.launch(share=True)

Mounted at /content/drive
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Epoch [1/5], Total Iterations: 100, Loss: 0.9760
Epoch [1/5], Total Iterations: 200, Loss: 0.1538
Epoch [1/5], Total Iterations: 300, Loss: 0.0715
Epoch [1/5], Total Iterations: 400, Loss: 0.0448
Epoch [1/5], Total Iterations: 500, Loss: 0.0316
Epoch [1/5], Total Iterations: 600, Loss: 0.0334
Epoch [2/5], Total Iterations: 700, Loss: 0.0137
Epoch [2/5], Total Iterations: 800, Loss: 0.0186
Epoch [2/5], Total Iterations: 900, Loss: 0.0100
Epoch [2/5], Total Iterations: 1000, Loss: 0.0077
Epoch [2/5], Total Iterations: 1100, Loss: 0.0107
Epoch [2/5], Total Iterations: 1200, Loss: 0.0130
Epoch [3/5], Total Iterations: 1300, Loss: 0.0098
Epoch [3/5], Total Iterations: 1400, Loss: 0.0057
Epoch [3/5], Total Iterations: 1500, Loss: 0.0102
Epoch [3/5], Total Iterations: 1600, Loss: 0.0097
Epoch [3/5], Total Iterations: 1700, Loss: 0.0058
Epoch [3/5], Total Iterations: 1800, Loss: 0.0089
Epoch [4/5], Total Iterations: 1900, Loss: 0.0046
Epoch [4/5], Total Iterations: 2000, Loss: 0.0018
Epoch [4/